In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from PIL import Image
from ultralytics import YOLO


IMG_SIZE = (224, 224)
USE_CHANNEL_LAST = False
DATASET = "laion"
DEVICE = "cuda:0"
PRESERVE_ASPECT_RATIO = False
SHIFT = 0

yolo_path = "./yolo/runs/detect/train_4class_noGbif/weights/best.pt"
yolo_model = YOLO(
    yolo_path,
    task="detect",
)


In [4]:
def get_bbox(img, yolo_model):
    results = yolo_model(img, verbose=False, device=DEVICE, max_det=1)
    img_w, img_h, _ = img.shape
    bbox = [0, 0, img_w, img_h]
    conf = 0.0
    
    for result in results:
        _bbox = [0, 0, img_w, img_h]
        _conf = 0.0

        bboxes_tmp = result.boxes.xyxy.tolist()
        confs_tmp = result.boxes.conf.tolist()

        for bbox_tmp, conf_tmp in zip(bboxes_tmp, confs_tmp):
            if conf_tmp > _conf:
                _bbox = bbox_tmp
                _conf = conf_tmp

        if _conf > conf:
            bbox = _bbox
            conf = _conf

    bbox = [int(float(mcb)) for mcb in bbox]

    return bbox, conf

def generate_df(dir_folder):
    images = []
    for class_folder in os.listdir(dir_folder):
        class_path = os.path.join(dir_folder, class_folder)
        if os.path.isdir(class_path):
            for img_name in os.listdir(class_path):
                if img_name.endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(class_path, img_name)
                    with Image.open(img_path) as img:
                        width, height = img.size
                        
                        bbx_xtl = 0
                        bbx_ytl = 0
                        bbx_xbr = width
                        bbx_ybr = height
                        
                        images.append([img_name, width, height, bbx_xtl, bbx_ytl, bbx_xbr, bbx_ybr, class_folder])
    
    df = pd.DataFrame(images, columns=['img_fName', 'img_w', 'img_h', 'bbx_xtl', 'bbx_ytl', 'bbx_xbr', 'bbx_ybr', 'class_label'])
    
    return df
        
def load_image(img_path):
    img = Image.open(img_path)
    img_rgb = img.convert("RGB")
    img_np = np.array(img_rgb)
    
    return img_np    

In [7]:
ina_root = '../inaturalist-six-cropped/inaturalist/inaturalist_crawler_data_all_cleaned_sub'
df = generate_df(ina_root)
df['img_fName'] = ina_root + '/' + df['class_label'] + '/' + df['img_fName']

# Bounding boxes
for idx, row in df.iterrows():
    img_path = row['img_fName']
    img = load_image(img_path)
    bbox, _ = get_bbox(img, yolo_model)

    df.at[idx, 'bbx_xtl'] = bbox[0] 
    df.at[idx, 'bbx_ytl'] = bbox[1] 
    df.at[idx, 'bbx_xbr'] = bbox[2] 
    df.at[idx, 'bbx_ybr'] = bbox[3]

destination = '../data_round_2'
df.to_csv(os.path.join(destination, 'inaturalist_anno.csv'), index=False)
    

,img_fName,img_w,img_h,bbx_xtl,bbx_ytl,bbx_xbr,bbx_ybr,class_label
0,../inaturalist-six-cropped/inaturalist/inatura...,1407,1377,0,0,1407,1377,japonicus
1,../inaturalist-six-cropped/inaturalist/inatura...,679,703,0,0,679,703,japonicus
2,../inaturalist-six-cropped/inaturalist/inatura...,695,948,0,0,695,948,japonicus
3,../inaturalist-six-cropped/inaturalist/inatura...,1068,1001,0,0,1068,1001,japonicus
4,../inaturalist-six-cropped/inaturalist/inatura...,1315,894,0,0,1315,894,japonicus
